In [ ]:
# import python libraries and nba library
import numpy as np
import nba_api as nba
from matplotlib import pyplot as plt
import pandas as pd
import socket
import logging
import os
import glob

In [ ]:
# Get all files in working extraction data directory
path = os.getcwd()
path = path.replace("notebooks","data")+'\\'+'extraction-files'
csv_files = glob.glob(os.path.join(path,"*.csv"))

In [ ]:
# bringing in all new files
extract_df = pd.DataFrame()
for file in csv_files:
    df = pd.read_csv(file)
    frames = [extract_df, df]
    extract_df = pd.concat(frames)

In [ ]:
# Cosmetic changes and dropping duplicaes that snuck in on import
extract_df = extract_df.rename(columns={"Unnamed: 0": "Season #"})
extract_df = extract_df.drop(columns=["LEAGUE_ID", "GS"])
extract_df = extract_df.drop_duplicates()

In [5]:
# Get all files in working evergreen data directory
path = os.getcwd()
path = path.replace("notebooks","data")+'\\'+'evergreen-files'
csv_files = glob.glob(os.path.join(path,"*.csv"))

In [6]:
# bring in evergreen file
evergreen_df = pd.DataFrame()
for file in csv_files:
    df = pd.read_csv(file)
    frames = [evergreen_df, df]
    evergreen_df = pd.concat(frames)

In [7]:
evergreen_df = evergreen_df.rename(columns={"Unnamed: 0": "Rank"})
evergreen_df = evergreen_df.drop_duplicates()
evergreen_df = evergreen_df.drop(columns=["Rank", "RANK",'EFF','AST_TOV','STL_TOV'])
evergreen_df.insert(0, "Season #", "Now")
evergreen_df.insert(3, "SEASON_ID", "2022-23")

In [8]:
extract_df = extract_df[(extract_df["SEASON_ID"] != "2022-23")]

In [9]:
# # MWAR Formula Coefficients:
# coFGM = 2
# coFGA = -1.2
# coFG3M = 1
# coFG3A = -0.4
# coFTM = 1
# coFTA = -1
# coDREB = 1.5
# coOREB = 2
# coAST = 2.5
# coTOV = -3
# coSTL = 2
# coBLK = 2
# coPF = -0.5
# coPTS = 1 

# totFGM = coFGM * full_df["FGM"]
# totFGA = coFGA * full_df["FGA"]
# totFG3M = coFG3M * full_df["FG3M"]
# totFG3A = coFG3A * full_df["FG3A"]
# totFTM = coFTM * full_df["FTM"]
# totFTA = coFTA * full_df["FTA"]
# totDREB = coDREB * full_df["DREB"]
# totOREB = coOREB * full_df["OREB"]
# totAST = coAST * full_df["AST"]
# totTOV = coTOV * full_df["TOV"]
# totSTL = coSTL * full_df["STL"]
# totBLK = coBLK * full_df["BLK"]
# totPF = coPF * full_df["PF"]
# totPTS = coPTS * full_df["PTS"]
# totMIN = full_df["MIN"] - 18

# full_df["MWAR"] = totFGM + totFGA + totFG3A + totFG3M + totFTM + totFTA + totDREB + totOREB + totAST + totTOV + totSTL + totBLK + totPF + totPTS + totMIN

In [10]:
full_df = pd.DataFrame(np.vstack([extract_df, evergreen_df]), columns=evergreen_df.columns)

In [11]:
FGMA = (full_df["FGA"] - full_df["FGM"])*1.25
FG3MA = (full_df["FG3A"] - full_df["FG3M"])*1.25
FTMA = (full_df["FTA"]-full_df["FTM"])*0.9
REB = 3*full_df["OREB"] + 2*full_df["DREB"]
ATR = 3*(full_df["AST"]-full_df["TOV"])
STB = 2*(full_df["BLK"]+full_df["STL"])
PF = full_df["PF"]*-0.5
PTS = full_df["PTS"]*1
MIN = full_df["MIN"] - 18*full_df["GP"]

full_df["MOWAR"] = FGMA + FG3MA + FTMA + PTS
full_df["MDWAR"] = STB + PF
full_df["MPWAR"] = ATR
full_df["MIWAR"] = REB + MIN
full_df["MWAR"] = FGMA + FG3MA + FTMA + REB + ATR + STB + PF + PTS + MIN
full_df["MWARPM"] = full_df["MWAR"]/full_df["MIN"]
full_df["MWARPG"] = full_df["MWAR"]/full_df["GP"]

In [12]:
# full_2021_test = full_df.iloc[:,[2,3,8,-8,-7,-6,-5,-4,-3,-2,-1]][(full_df["SEASON_ID"] == "2022-23") & (full_df["GP"] > 20)].sort_values("MWARPM", ascending = False)

In [14]:
file_path = "C:/Github Repo/Mario-Jupyter-Codespace/data/basketballProject/analysis-files/"
# file_path = "../../data/basketballProject/analysis-files/"
file_name = "consolidated_evergreen_stats"
file_extension = ".csv"
full_path = file_path + file_name + file_extension
full_df.to_csv(full_path)